In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define the ticker symbols
tickers = {
    'NIFTY50': '^NSEI',  # Yahoo Finance symbol for NIFTY 50 Index
    'TATA_MOTORS': 'TATAMOTORS.NS'  # Yahoo Finance symbol for TATA MOTORS (NSE)
}

# Define the date range
start_date = '2023-04-01'
end_date = '2024-04-01'

# Risk-free rate (annualized)
risk_free_rate = 7.015 / 100

# Download data for each ticker
data = {}
for name, ticker in tickers.items():
    print(f"Downloading data for {name} ({ticker})...")
    data[name] = yf.download(ticker, start=start_date, end=end_date)

# Calculate daily returns
returns = {}
for name, df in data.items():
    df['Daily Return'] = df['Adj Close'].pct_change()
    returns[name] = df['Daily Return'].dropna()

# Calculate alpha, beta, sharpe ratio, and max drawdown for TATA MOTORS with respect to NIFTY50
nifty_returns = returns['NIFTY50']
tata_returns = returns['TATA_MOTORS']

# Covariance and variance for beta calculation
cov_matrix = np.cov(tata_returns, nifty_returns)
beta = cov_matrix[0, 1] / cov_matrix[1, 1]
alpha = tata_returns.mean() - beta * nifty_returns.mean()

# Sharpe ratio calculation (annualized)
annual_factor = np.sqrt(252)
sharpe_ratios = {}
for name, ret in returns.items():
    excess_return = ret - (risk_free_rate / 252)
    sharpe_ratios[name] = (excess_return.mean() / excess_return.std()) * annual_factor

# Max drawdown calculation (O(n))
def calculate_max_drawdown(prices):
    max_drawdown = 0
    peak = prices[0]
    for price in prices:
        if price > peak:
            peak = price
        drawdown = (peak - price) / peak
        max_drawdown = max(max_drawdown, drawdown)
    return max_drawdown

max_drawdowns = {}
for name, df in data.items():
    max_drawdowns[name] = calculate_max_drawdown(df['Adj Close'].dropna())

# Print results
print("Alpha (TATA MOTORS):", alpha)
print("Beta (TATA MOTORS):", beta)
for name in tickers:
    print(f"Sharpe Ratio ({name}): {sharpe_ratios[name]:.2f}")
    print(f"Max Drawdown ({name}): {max_drawdowns[name]:.2%}")




[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed

Alpha (TATA MOTORS): 0.002494182222248188
Beta (TATA MOTORS): 1.0940495680506144
Sharpe Ratio (NIFTY50): 1.96
Max Drawdown (NIFTY50): 6.61%
Sharpe Ratio (TATA_MOTORS): 3.56
Max Drawdown (TATA_MOTORS): 9.51%



/var/folders/qh/vbb1qm9s49s09w22cg64fbnw0000gn/T/ipykernel_54516/1387876436.py:49: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  peak = prices[0]
